In [25]:
import numpy as np
import pandas as pd

#Neural Network Classifier
class NN(object):
    def __init__(self, layers, biases, weights):
            self.num_layers = len(layers)
            self.sizes = layers
            self.biases = biases
            self.weights = []
            count = 0
            for i in range(len(layers) - 1):
                l1 = self.sizes[i]
                l2 = self.sizes[i+1]
                temp_array = np.array(weights[count: count+l1*l2])
                temp_array.shape = (l2, l1)
#                 print(i, temp_array)
                self.weights.append(temp_array)
                count += l1*l2
#             print("self.weights = " + str(self.weights))

    #backpropagation algorithm
    def backprop(self, x, y):
        update_b = [np.zeros(b.shape) for b in self.biases]
        update_w = [np.zeros(w.shape) for w in self.weights]
        activation = np.array(x)

        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = (np.dot(w, activation))
            z.shape = (w.shape[0], 1)
            # print("z.shape = ", z.shape)
            z = z+b;
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        delta = self.cd(activations[-1], y)*sd(zs[-1])
        update_b[-1] = delta
        update_w[-1] = np.dot(delta, activations[-2].transpose())
        activations[0].shape = (self.sizes[0], 1)

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sd(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta)*sp
            update_b[-l] = delta
            update_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return update_b, update_w

    #accuracy calculation
    def accuracy(self, X, Y, Y_Train, L = 30):
        y = self.fprop(X)
#         print("Forward propagation results = " + str(y))
#         y = [1 if p>0.5 else 0 for p in y[0]]
        calc_fp = [((p - self.yes_mean)/self.yes_stddev, (1-p-self.no_mean)/self.no_stddev) for p in y[0] ]
        similarity_array = []
        for i in range(len(calc_fp)):
            similarity_array.append([])
        
        h_array_test = []
        for i in range(len(calc_fp)):
            point = np.array(calc_fp[i])
            dist = np.linalg.norm(point - self.origin)
            calc_value = (1 - np.exp(-dist/2)) / (dist * (1 + np.exp(-dist/2)))
            h_array_test.append((calc_fp[i][0]*calc_value, calc_fp[i][1]*calc_value))
            
        for i in range(len(similarity_array)):
            for j in range(len(self.h_array)):
                dist = np.linalg.norm(np.array(h_array_test[i]) - np.array(self.h_array[j]))
                similarity_array[i].append((dist, j))
                
            similarity_array[i] = sorted(similarity_array[i])
            
        y = []
            
        for i in range(len(calc_fp)):
            ones = 0
            zeroes = 0
            for j in range(L):
                if (Y_Train[similarity_array[i][j][1]] == 1):
                    ones += 1
                else:
                    zeroes += 1
                    
            if (ones >= zeroes):
                y.append(1)
            else:
                y.append(0)
        
        acc = sum([1 if f==F else 0 for f, F in zip(y, Y)])
        tp = 0
        fp = 0
        fn = 0
        for i, j in zip(y, Y):
            if i==1 and j==1:
                tp=tp+1
            if i==1 and j==0:
                fp=fp+1
            if i==0 and j==1:
                fn=fn+1
        precision=0
        try:
            precision = float(tp)/(tp+fp)
        except Exception as e:
            print(e)
        recall=0
        try:
            recall = float(tp)/(tp+fn)
        except Exception as e:
            print(e)
        
        return float(acc)/10.0, precision, recall

    #forward propagation
    def fprop(self, a):
        a = a.transpose()
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    #Stochastic GD : one record at a time
    def Stoch_GD(self, X, Y, alpha, i):
            for _ in range(1):
                for x,y in zip(X, Y):
                    delta_update_b, delta_update_w = self.backprop(x, y)
                    self.weights = [w-alpha*nw for w, nw in zip(self.weights, delta_update_w)]
                    self.biases = [b-alpha*nb for b, nb in zip(self.biases, delta_update_b)]
                    
                    
            #now forward propagate to obtain the actual prediction results
            res = self.fprop(X)
            
            #IRIS-Versicolor data
            self.yes = [y for y in res[0]]
            self.yes_mean = np.mean(self.yes)
            self.yes_stddev = np.std(self.yes)
            
            #IRIS-Setosa data
            self.no = [1-y for y in res[0]]
            self.no_mean = np.mean(self.no)
            self.no_stddev = np.std(self.no)
            
            #first element in tuple is for IRIS-Versicolor and second is for IRIS-Setosa
            self.predictions = [((y-self.yes_mean)/self.yes_stddev, (1-y-self.no_mean)/self.no_stddev) for y in res[0]]
            self.origin = np.array((0, 0))
            
            self.h_array = []
            for i in range(len(self.predictions)):
                point = np.array(self.predictions[i])
                dist = np.linalg.norm(point - self.origin)
                calc_value = (1 - np.exp(-dist/2)) / (dist * (1 + np.exp(-dist/2)))
                self.h_array.append((self.predictions[i][0]*calc_value, self.predictions[i][1]*calc_value))
                
                
#             print("self.h_array = " + str(self.h_array))
            self.sim_array = []
            for i in range(len(self.predictions)):
                self.sim_array.append([])
            
            for i in range(len(self.h_array)):
                for j in range(len(self.h_array)):
                    if (i != j):
                        dist = 2 - np.linalg.norm(np.array(self.h_array[i]) - np.array(self.h_array[j]))
                        self.sim_array[i].append((dist, j))
                        
                self.sim_array[i] = sorted(self.sim_array[i])
                        
            
#             print("self.sim_array = " + str(self.sim_array))
            
    
    #Calculating the optimisation function
    def optimisation_func(self, y, L=30, alpha=0.01):
        self.opt_value = 0
        for i in range(len(self.sim_array)):
            self_val = 0
            non_self_val = 0
            for j in range(L):
                if (y[i] == y[self.sim_array[i][j][1]]):
                    self_val += self.sim_array[i][j][0]
                else:
                    non_self_val += self.sim_array[i][j][0]
            
            self.opt_value += non_self_val - alpha*self_val
            
        return self.opt_value
            

    #difference in output
    def cd(self, output, y):
        return output-y

    #return the weights and biases
    def get_weights_and_biases(self):
        return self.weights, self.biases

def getData():
    df = pd.read_csv('IRIS.csv')
    df = df.sample(frac=1)
    X = df.as_matrix()
    y = X[:, -1]
    y = [0 if x == 'Iris-setosa'  else 1 for x in y]
    X = X[:, 0:-1]
    return X, y

def sigmoid(z):
    z = z.astype(float)
    return 1.0/(1.0+np.exp(-z))


def sd(z):
    return sigmoid(z)*(1-sigmoid(z))

# Selecting the parents that will procreate to give the new generation
def selection(ranks, population):
    sorted(ranks, key = lambda ranks: ranks[0])
    new_population = []
    ranks = ranks[:5]
    temp_pop = []
    for i in range(len(ranks)):
        temp_pop.append(population[ranks[i][1]])
    for i in range(0, 6):
        new_population += temp_pop

    return new_population

#CrossOver in the population : Swapping the first and last halves of the two parents
def crossover(population, c_rate):
    l = len(population)
    num_cross_overs = (int)(l / c_rate)
    for i in range(num_cross_overs):
        l1 = (int)(np.random.rand()*l - 1)
        l2 = (int)(np.random.rand()*l - 1)
        parent1 = population[l1]
        parent2 = population[l2]
        parent1 = parent1.transpose()
        parent2 = parent2.transpose()
        parent1 = parent1[0].tolist()
        parent2 = parent2[0].tolist()
#         print("Parent 1 = " + str(parent1))
#         print("Parent 2 = " + str(parent2))
        l = len(parent1)
        l = (int)(l/2)
#         print("Parent1[:l] = " + str(parent1[:l]))
#         print("Parent2[l:] = " + str(parent2[l:]))
        child = parent1[:l] + parent2[l:]
        
#         print("Length Child = " + str(len(child)))
        child = np.array(child)
        child.shape = (25, 1)
#         print("Child = " + str(child))
        population.append(child)
        
    return population

def k_fold_algo(k, X, y):
    # print("y = ", y)
    test = []
    train = []
    test_y = []
    train_y = []
    for i in range(0, len(X)):
        if i%10==k:
            test.append(X[i])
            test_y.append(y[i])
        else:
            train.append(X[i])
            train_y.append(y[i])

    return np.array(train), np.array(test), np.array(train_y), np.array(test_y)

if __name__ == '__main__':
        X, y = getData()
#         print("y = ", y)
        layers = [4,5,1]
        tot = 0
        for i in range(len(layers) - 1):
            l1 = layers[i]
            l2 = layers[i+1]
            tot += l1*l2
        
        weights_array = []
        biases_array = []
        for i in range(0, 30):
            biases = [np.random.randn(l, 1) for l in layers[1:]]
            weights = np.random.randn(tot, 1)
            weights_array.append(weights)
            biases_array.append(biases)
        
        rank_list = []
        for l in range(0,10):
#             print("l = " + str(l))
            rank_list = []
            accuracy_sum = 0
            for i in range(0, 30):
#                 print("i = " + str(i))
    #             print("Iteration #" + str(i))
                biases = biases_array[i]
                weights = np.array(weights_array[i], copy=True)
    #             print("Weights array = " + str(weights))
                net  = NN([4,5, 1], biases, weights)
                # print "Initial Random Bias:"+str(net.biases)
                # print "Initial Random Weights:"+str(net.weights)
                X_train, X_test, y_train, y_test = k_fold_algo(0, X, y)
    #             for k in range(2):
                net.Stoch_GD(X_train, y_train, 0.1, i)
                opt_value = net.optimisation_func(y_train)
#                 print("Opt_value = " + str(opt_value))
                answers = net.accuracy(X_test, y_test, y_train)
    #             print("answers = " + str(answers))
                accuracy_sum = accuracy_sum+answers[0]
                rank_list.append((opt_value, i))
                weights, biases = net.get_weights_and_biases()
                temp = []
                for j in range(len(layers)-1):
                    weights1 = np.array(weights[j], copy=True)
                    weights1 = weights1.reshape((layers[j+1]*layers[j], 1))
                    temp.append(weights1)
                temp = np.vstack((temp[0], temp[1]))
    #             print("temp = " + str(temp))
                weights_array[i] = temp
                biases_array[i] = biases
    #             print("Accuracy "+str(i+1)+" : "+str(answers[0]))
    #             print("Precision is: "+str(answers[1]))
    #             print("Recall is:"+str(answers[2]))
    #         for i in range(len(weights_array)):
    #         temp = weights_array[0]
    #         print(temp.transpose())
    #         temp = np.array(temp)
    #         temp = temp.reshape((1, tot))
    #         print("Demo = " + str(temp))
            new_pop = selection(rank_list, weights_array)
            new_pop = crossover(weights_array, 0.25)
            new_pop = new_pop[-30:]
            weights_array = new_pop
#             print("New Population = " + str(new_pop))
    #             print("Len New Pop = " + str(len(new_pop)))
            print("Avg Accuracy for iteration #{}: {}".format(l, accuracy_sum/30.0))
        
        rank_list = sorted(rank_list)
        index = rank_list[0][1]
        biases = biases_array[i]
        weights = weights_array[i]
        net = NN([4, 5, 1], biases, weights)
        X_train, X_test, y_train, y_test = k_fold_algo(0, X, y)
        net.Stoch_GD(X_train, y_train, 0.1, i)
        opt_value = net.optimisation_func(y_train)
        answers = net.accuracy(X_test, y_test, y_train)
        print("Final Answers(acc, prec, recall) = " + str(answers))
        


Avg Accuracy for iteration #0: 0.99
Avg Accuracy for iteration #1: 0.9633333333333334
Avg Accuracy for iteration #2: 0.9933333333333334
Avg Accuracy for iteration #3: 1.0
Avg Accuracy for iteration #4: 1.0
Avg Accuracy for iteration #5: 1.0
Avg Accuracy for iteration #6: 1.0
Avg Accuracy for iteration #7: 1.0
Avg Accuracy for iteration #8: 1.0
Avg Accuracy for iteration #9: 1.0
Final Answers(acc, prec, recall) = (1.0, 1.0, 1.0)
